In [25]:
import pandas as pd
import re
from pathlib import Path
import datetime
from dataclasses import dataclass

In [6]:
log_path = Path("../pager.log")
date_re = re.compile(" (\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}):")
flex_re = re.compile("FLEX_NEXT\|(\d+)\/(\d+)\|(\d+\.\d+\.\w)\|(\d+)\|(\w{2})\|(\d)\|(\w+)\|(\d)\.(\d)\.(K|F|C)\|(.*)")

In [13]:
@dataclass
class FragmentItem:
    text: str
    time: float

In [90]:
fragment_messages = {}
with open(log_path) as f:
    for line in f:
        date_match = date_re.search(line)
        flex_match = flex_re.search(line)
        
        if flex_match is None: continue
        if date_match is None: continue
        try: timestamp = datetime.strptime(date_match[1], "%Y-%m-%d %H:%M:%S").timestamp()
        except: continue

        frag_flag = flex_match[10]
        address = flex_match[4]
        text = flex_match[11]

        if frag_flag == "K": continue

        elif frag_flag == "F":
            fragment_messages[address] = FragmentItem(text=text, time=timestamp) # This will overwite any previous incomplete messages to the address
            
        elif frag_flag == "C":
            if not address in fragment_messages: # Ignore C's without a previous F
                continue 
            else:
                fragment_messages[address].text += text
                # print("[DEFRAG]", fragment_messages[address].text)
                del fragment_messages[address]

        # Clean stale fragments to avoid memory leak
        for addr, frag_item in list(fragment_messages.items()):
            if timestamp - frag_item.time > 10: del fragment_messages[addr]        

